In [ ]:
# -*- coding: utf-8 -*-
import cv2 as cv
import numpy as np
import math
import mediapipe as mp
from PIL import Image, ImageDraw, ImageFont
import os

# ===== 설정 =====
CAM_ID = 0
SMOOTH_ALPHA = 0.2     # 각도 EMA 스무딩
SHOW_GUIDE = True

# ===== 색상 (BGR) =====
RED    = (0, 0, 255)      # 눈-눈 벡터 (요청: 빨강)
BLUE   = (255, 0, 0)      # 수평 기준선 (요청: 파랑)
GREEN  = (0, 200, 0)      # 각도 호
PURPLE = (128, 0, 128)     # 점선 보조
WHITE  = (255, 255, 255)  # 일반 텍스트
YELLOW = (0, 255, 255)    # 안내 텍스트(가독성↑)

# ===== 한글 폰트 탐색 =====
FONT_CANDIDATES = [
    "C:/Windows/Fonts/malgun.ttf",                       # Windows
    "/System/Library/Fonts/AppleGothic.ttf",             # macOS
    "/usr/share/fonts/truetype/nanum/NanumGothic.ttf",   # Ubuntu
    "/usr/share/fonts/truetype/noto/NotoSansCJK-Regular.ttc",
]
FONT_PATH = None
for fp in FONT_CANDIDATES:
    if os.path.exists(fp):
        FONT_PATH = fp
        break

def pil_put_text(img_bgr, text, org_xy, font_size=20, text_color=(255,255,0),
                 outline=True, bg_rect=False, bg_color=(0,0,0), bg_pad=6, weight=2):
    """
    OpenCV 이미지(BGR)에 PIL로 한글 텍스트 그리기.
    - text_color: RGB 기준 (내부에서 변환)
    - org_xy: 좌하단 기준 좌표(x, y)
    """
    if FONT_PATH is None:
        # 폰트가 없으면 OpenCV 기본 폰트로 영문 대체 (한글은 깨질 수 있음)
        cv.putText(img_bgr, text, org_xy, cv.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,255), 2, cv.LINE_AA)
        return img_bgr

    # BGR -> RGB -> PIL
    img_rgb = cv.cvtColor(img_bgr, cv.COLOR_BGR2RGB)
    pil_img = Image.fromarray(img_rgb)
    draw = ImageDraw.Draw(pil_img)

    try:
        font = ImageFont.truetype(FONT_PATH, font_size)
    except Exception:
        font = ImageFont.load_default()

    # 텍스트 크기 계산
    tw, th = draw.textbbox((0,0), text, font=font)[2:]
    x, y = org_xy
    # OpenCV는 좌하단 기준, PIL은 좌상단 기준 → 보정
    y_top = y - th

    # 배경 사각형 옵션
    if bg_rect:
        draw.rectangle((x - bg_pad, y_top - bg_pad, x + tw + bg_pad, y + bg_pad),
                       fill=bg_color)

    # 외곽선(가독성)
    if outline:
        for dx, dy in [(-1,0),(1,0),(0,-1),(0,1),(-1,-1),(1,-1),(-1,1),(1,1)]:
            draw.text((x+dx*weight, y_top+dy*weight), text, font=font, fill=(0,0,0))

    # 본문
    draw.text((x, y_top), text, font=font, fill=(text_color[0], text_color[1], text_color[2]))

    # PIL -> OpenCV(BGR)
    return cv.cvtColor(np.array(pil_img), cv.COLOR_RGB2BGR)

# ===== MediaPipe =====
mp_face_mesh = mp.solutions.face_mesh
mesh = mp_face_mesh.FaceMesh(
    static_image_mode=False,
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5,
)

LEFT_EYE_OUTER  = 33
RIGHT_EYE_OUTER = 263

def get_roll_and_pts(landmarks, w, h):
    lx = landmarks[LEFT_EYE_OUTER].x * w; ly = landmarks[LEFT_EYE_OUTER].y * h
    rx = landmarks[RIGHT_EYE_OUTER].x * w; ry = landmarks[RIGHT_EYE_OUTER].y * h
    dx, dy = (rx - lx), (ry - ly)
    ang = math.degrees(math.atan2(dy, dx))
    return ang, (int(lx), int(ly)), (int(rx), int(ry)), math.hypot(dx, dy)

def ema(prev, new, a):
    return new if prev is None else (a*new + (1-a)*prev)

def draw_dashed_line(img, pt1, pt2, color, thickness=1, dash_len=8, gap_len=6):
    x1, y1 = pt1; x2, y2 = pt2
    dist = int(math.hypot(x2-x1, y2-y1))
    if dist == 0: return
    for i in range(0, dist, dash_len + gap_len):
        t1 = i / dist
        t2 = min(i + dash_len, dist) / dist
        sx = int(x1 + (x2-x1) * t1); sy = int(y1 + (y2-y1) * t1)
        ex = int(x1 + (x2-x1) * t2); ey = int(y1 + (y2-y1) * t2)
        cv.line(img, (sx, sy), (ex, ey), color, thickness, cv.LINE_AA)

def draw_angle_arc(img, center, angle_deg, radius=42, color=GREEN, thickness=2):
    start = 0
    end   = int(angle_deg)
    if angle_deg < 0:
        start, end = int(angle_deg), 0
    cv.ellipse(img, center, (radius, radius), 0, start, end, color, thickness, cv.LINE_AA)

def main():
    cap = cv.VideoCapture(CAM_ID)
    if not cap.isOpened():
        print("웹캠 열기 실패")
        return

    roll_ema = None
    snap_mode = False  # 'H' 키로 안내선 숨김 토글 (캡처용)

    while True:
        ok, frame = cap.read()
        if not ok:
            break

        h, w = frame.shape[:2]
        rgb = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
        res = mesh.process(rgb)

        if res.multi_face_landmarks:
            lm = res.multi_face_landmarks[0].landmark
            roll_deg, pL, pR, eye_dist = get_roll_and_pts(lm, w, h)
            roll_ema = ema(roll_ema, roll_deg, SMOOTH_ALPHA)

            # === 시각 오버레이 ===
            if SHOW_GUIDE and not snap_mode:
                # 1) 수평 기준선 (요청: 파란색)
                cv.line(frame, (0, h//2), (w, h//2), BLUE, 2, cv.LINE_AA)

                # 2) 눈-눈 벡터 (요청: 빨강, 화살표)
                cv.arrowedLine(frame, pL, pR, RED, 3, cv.LINE_AA, tipLength=0.06)

                # 3) 롤 각도 호 (녹색)
                draw_angle_arc(frame, pL, roll_ema if roll_ema is not None else roll_deg,
                               radius=46, color=GREEN, thickness=2)

                # 4) 수직 투영 점선 (주황) — 기울어진 dy를 직관화
                proj = (pR[0], pL[1])
                draw_dashed_line(frame, pR, proj, PURPLE, 2, 8, 6)
                cv.circle(frame, proj, 4, PURPLE, -1, cv.LINE_AA)

                # ===== 한글 라벨 (PIL로 가독성 높임) =====
                dy = pR[1] - pL[1]
                text1 = f"롤 각도: {roll_deg:+.1f}도  (EMA {roll_ema:+.1f}도)" if roll_ema is not None else f"롤 각도: {roll_deg:+.1f}도"
                text2 = f"눈-눈 길이: {eye_dist:.1f}px   세로차(dy): {dy:+d}px"

                frame = pil_put_text(frame, text1, (12, 34), font_size=22, text_color=(255,255,0), bg_rect=True)
                frame = pil_put_text(frame, text2, (12, 66), font_size=20, text_color=(255,255,255), bg_rect=True)

                # 범례(한글)
                frame = pil_put_text(frame, "빨강: 눈-눈 벡터(기울기)", (12, 98), font_size=20, text_color=(255,100,100), bg_rect=True)
                frame = pil_put_text(frame, "파랑: 수평 기준선",       (12, 128), font_size=20, text_color=(135,180,255), bg_rect=True)
                frame = pil_put_text(frame, "녹색: 롤 각도(호로 표시)", (12, 158), font_size=20, text_color=(150,255,150), bg_rect=True)
                frame = pil_put_text(frame, "보라 점선: 수직 투영선(dy)", (12, 188), font_size=20, text_color=(180,100,255), bg_rect=True)

            else:
                # 캡처용: 최소 요소만 남김
                cv.arrowedLine(frame, pL, pR, RED, 3, cv.LINE_AA, tipLength=0.06)
                text = f"롤 각도: {roll_deg:+.1f}도"
                frame = pil_put_text(frame, text, (12, 34), font_size=22, text_color=(255,255,0), bg_rect=True)

        else:
            frame = pil_put_text(frame, "얼굴을 찾지 못했습니다", (12, 34), font_size=20, text_color=(255,200,120), bg_rect=True)

        # 하단 도움말
        help_text = "[S] 캡처 저장   [H] 가이드 토글   [ESC] 종료"
        (tw, th), _ = cv.getTextSize(help_text, cv.FONT_HERSHEY_SIMPLEX, 0.55, 1)
        cv.putText(frame, help_text, (w - tw - 10, h - 10), cv.FONT_HERSHEY_SIMPLEX, 0.55, WHITE, 1, cv.LINE_AA)

        cv.imshow("눈-눈 벡터 시각화 (Roll 이해용)", frame)
        key = cv.waitKey(1) & 0xFF
        if key == 27:
            break
        elif key in (ord('h'), ord('H')):
            snap_mode = not snap_mode
        elif key in (ord('s'), ord('S')):
            cv.imwrite("capture_eye_vector_kr.png", frame)
            print("저장됨: capture_eye_vector_kr.png")

    cap.release()
    cv.destroyAllWindows()

if __name__ == "__main__":
    main()


In [10]:
# -*- coding: utf-8 -*-
import cv2 as cv
import numpy as np
import math
import mediapipe as mp
from PIL import Image, ImageDraw, ImageFont
import os

# ===== 설정 =====
CAM_ID = 0
SMOOTH_ALPHA = 0.25     # 각도 EMA 스무딩
SHOW_GUIDE = True

# ===== 색상 (BGR) =====
MAGENTA = (255, 0, 255)   # 코-턱 벡터 (마젠타)
WHITE   = (255, 255, 255) # 세로 기준선 (흰색)
BLUE    = (255, 0, 0)     # 수평 투영 점선 (파랑)

# ===== 한글 폰트 탐색 =====
FONT_CANDIDATES = [
    "C:/Windows/Fonts/malgun.ttf",                       # Windows
    "/System/Library/Fonts/AppleGothic.ttf",             # macOS
    "/usr/share/fonts/truetype/nanum/NanumGothic.ttf",   # Ubuntu
    "/usr/share/fonts/truetype/noto/NotoSansCJK-Regular.ttc",
]
FONT_PATH = None
for fp in FONT_CANDIDATES:
    if os.path.exists(fp):
        FONT_PATH = fp
        break

def pil_put_text(img_bgr, text, org_xy, font_size=20, text_color=(255,255,0),
                 outline=True, bg_rect=False, bg_color=(0,0,0), bg_pad=6, weight=2):
    """ OpenCV(BGR) 이미지 위에 PIL로 한글 텍스트(가독성↑) """
    if FONT_PATH is None:
        cv.putText(img_bgr, text, org_xy, cv.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,255), 2, cv.LINE_AA)
        return img_bgr

    img_rgb = cv.cvtColor(img_bgr, cv.COLOR_BGR2RGB)
    pil_img = Image.fromarray(img_rgb)
    draw = ImageDraw.Draw(pil_img)

    try:
        font = ImageFont.truetype(FONT_PATH, font_size)
    except Exception:
        font = ImageFont.load_default()

    # 텍스트 크기
    tw, th = draw.textbbox((0,0), text, font=font)[2:]
    x, y = org_xy
    y_top = y - th  # OpenCV(좌하단) vs PIL(좌상단) 기준 보정

    if bg_rect:
        draw.rectangle((x - bg_pad, y_top - bg_pad, x + tw + bg_pad, y + bg_pad),
                       fill=bg_color)
    if outline:
        for dx, dy in [(-1,0),(1,0),(0,-1),(0,1),(-1,-1),(1,-1),(-1,1),(1,1)]:
            draw.text((x+dx*weight, y_top+dy*weight), text, font=font, fill=(0,0,0))
    draw.text((x, y_top), text, font=font, fill=text_color)

    return cv.cvtColor(np.array(pil_img), cv.COLOR_RGB2BGR)

# ===== MediaPipe FaceMesh =====
mp_face_mesh = mp.solutions.face_mesh
mesh = mp_face_mesh.FaceMesh(
    static_image_mode=False,
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5,
)

# 랜드마크 인덱스 (MediaPipe FaceMesh)
NOSE_TIP = 4    # 코끝
CHIN     = 152  # 턱끝

def get_pitch_and_pts(landmarks, w, h):
    # 좌표 픽셀화
    nx = landmarks[NOSE_TIP].x * w; ny = landmarks[NOSE_TIP].y * h
    cx = landmarks[CHIN].x     * w; cy = landmarks[CHIN].y     * h
    nx_i, ny_i = int(nx), int(ny)
    cx_i, cy_i = int(cx), int(cy)

    # 코→턱 벡터
    dx, dy = (cx - nx), (cy - ny)

    # x축(수평) 기준 각도 (deg). 영상 좌표는 y↓가 + 이므로 아래쪽이 +각
    angle_deg = math.degrees(math.atan2(dy, dx))

    # "세로축(수직)" 기준에서의 편차를 피치 각도로 정의: 90°가 세로
    pitch_deg = angle_deg - 90.0  # 정면에선 대략 0° 근처

    length_px = math.hypot(dx, dy)
    return pitch_deg, (nx_i, ny_i), (cx_i, cy_i), length_px, angle_deg  # angle_deg도 반환(호 표시에 사용)

def ema(prev, new, a):
    return new if prev is None else (a*new + (1-a)*prev)

def draw_dashed_line(img, pt1, pt2, color, thickness=2, dash_len=8, gap_len=6):
    x1, y1 = pt1; x2, y2 = pt2
    dist = int(math.hypot(x2-x1, y2-y1))
    if dist == 0: return
    for i in range(0, dist, dash_len + gap_len):
        t1 = i / dist
        t2 = min(i + dash_len, dist) / dist
        sx = int(x1 + (x2-x1) * t1); sy = int(y1 + (y2-y1) * t1)
        ex = int(x1 + (x2-x1) * t2); ey = int(y1 + (y2-y1) * t2)
        cv.line(img, (sx, sy), (ex, ey), color, thickness, cv.LINE_AA)



def main():
    cap = cv.VideoCapture(CAM_ID)
    if not cap.isOpened():
        print("웹캠 열기 실패")
        return

    pitch_ema = None
    snap_mode = False  # 'H' 키로 안내선 숨김 토글 (캡처용)

    while True:
        ok, frame = cap.read()
        if not ok:
            break

        h, w = frame.shape[:2]
        rgb = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
        res = mesh.process(rgb)

        if res.multi_face_landmarks:
            lm = res.multi_face_landmarks[0].landmark
            pitch_deg, pN, pC, nose_chin_len, angle_from_x = get_pitch_and_pts(lm, w, h)
            pitch_ema = ema(pitch_ema, pitch_deg, SMOOTH_ALPHA)

            if SHOW_GUIDE and not snap_mode:
                # 1) 세로 기준선 (코끝을 지나게 그려, 세로축 기준을 직관화)
                cv.line(frame, (pN[0], 0), (pN[0], h), WHITE, 2, cv.LINE_AA)

                # 2) 코-턱 벡터 (마젠타)
                cv.arrowedLine(frame, pN, pC, MAGENTA, 4, cv.LINE_AA, tipLength=0.06)


                # 4) 수평 투영 점선(파랑) — 턱을 코끝의 x와 동일한 수직선까지 가로로 투영
                proj = (pN[0], pC[1])  # 코끝 x, 턱 y
                draw_dashed_line(frame, pC, proj, BLUE, 2, 8, 6)
                cv.circle(frame, proj, 5, BLUE, -1, cv.LINE_AA)

                # ===== 한글 라벨 =====
            
                text2 = f"코-턱 길이: {nose_chin_len:.1f}px   가로 오프셋(dx): {pC[0]-pN[0]:+d}px"

                
                frame = pil_put_text(frame, text2, (12, 66), font_size=20, text_color=(255,255,255), bg_rect=True)

                # 범례
                frame = pil_put_text(frame, "마젠타: 코-턱 벡터(숙임/젖힘 지표)", (12, 98),  font_size=20, text_color=(255,120,255), bg_rect=True)
                frame = pil_put_text(frame, "흰색: 세로 기준선(코끝 통과)",       (12, 128), font_size=20, text_color=(255,255,255), bg_rect=True)
                frame = pil_put_text(frame, "녹색: 피치 각도(호로 표시)",         (12, 158), font_size=20, text_color=(150,255,150), bg_rect=True)
                frame = pil_put_text(frame, "파랑 점선: 수평 투영선(dx)",         (12, 188), font_size=20, text_color=(135,180,255), bg_rect=True)

            else:
                # 캡처용 최소 화면
                cv.arrowedLine(frame, pN, pC, MAGENTA, 4, cv.LINE_AA, tipLength=0.06)
                text = f"피치 각도: {pitch_deg:+.1f}도"
                frame = pil_put_text(frame, text, (12, 34), font_size=22, text_color=(255,255,0), bg_rect=True)

        else:
            frame = pil_put_text(frame, "얼굴을 찾지 못했습니다", (12, 34), font_size=20, text_color=(255,200,120), bg_rect=True)

        # 하단 도움말
        help_text = "[S] 캡처 저장   [H] 가이드 토글   [ESC] 종료"
        (tw, th), _ = cv.getTextSize(help_text, cv.FONT_HERSHEY_SIMPLEX, 0.55, 1)
        cv.putText(frame, help_text, (w - tw - 10, h - 10), cv.FONT_HERSHEY_SIMPLEX, 0.55, WHITE, 1, cv.LINE_AA)

        cv.imshow("코-턱 벡터 시각화 (Pitch 이해용)", frame)
        key = cv.waitKey(1) & 0xFF
        if key == 27:
            break
        elif key in (ord('h'), ord('H')):
            snap_mode = not snap_mode
        elif key in (ord('s'), ord('S')):
            cv.imwrite("capture_nose_chin_pitch_kr.png", frame)
            print("저장됨: capture_nose_chin_pitch_kr.png")

    cap.release()
    cv.destroyAllWindows()

if __name__ == "__main__":
    main()
